# EVOLUCIÓN BICIMAD (2019-2023)
## Un trabajo de: Gabriela Brito Santana y Francisco Suraña Muñoz
##### Este trabajo de análisis de grandes cantidades de datos mediante Spark se centra en el estudio de la evolución del uso del servicio de bicicleta pública eléctrica de Madrid (Bicimad) entre 2019 y 2023. Para ello se van a analizar los datos del mes de enero de cada uno de los años que son objeto del estudio. Las cifras consultadas son las recogidas por la EMT, a las que se accede a través de su portal de datos abiertos.
##### En esta práctica se hace una comparativa  a lo largo de los años en el número de desplazamientos durante dicho mes, así como, las estaciones de servicio con mayor y menor uso.
##### Por otra parte, cabe destacar que a partir del año 2022 se produce un cambio en la forma de recoger y exponer los datos, por este motivo, se va a hacer un estudio de algunos datos por separado. Para los tres primeros años se observará el tipo de usuario y el rango de edad que más utiliza el servicio, además del tiempo de viaje, mientras que, para los dos últimos años, se analizarán cuáles son los días del mes con mayor número de viajes.
##### Finalmente, se hará un pequeño análisis de los datos durante los primeros meses de la pandemia donde se observará de forma evidente los estragos del confinamiento y la reducción de movimiento que se produjo.

##### Los datos ofrecidos por la EMT pueden contener la siguiente información:
##### Para los archivos en formato json
 - id: Identificación del movimiento
 - user_day_code: Código del ususario. Para una misma fecha, todos los movimeintos del mismo ususario tendrán el mismo código.
 - user_type: Número que indica el tipo de ususario que ha realizado el movimiento. Sus posibles valores son: 
    -   0: No se ha podido determinar el tipo de ususario 
    -    1: Usuario anual
    -    2: Usuario ocasional
    -    3: Trabajador de la empresa
 - ageRange: Núemro que indica el ragno de edad del ususario que ha realizado el movimiento. Sus posibles valores son:
   - 0: No se ha podido determinar el rango de edad del usuario
   -    1: El ususario tiene entre 0 y 16 años
   -    2: El ususario tiene entre 17 y 18 años
   -    3: El usuario tiene entre 19 y 26 años
   -    4: El usuario tiene entre 27 y 40 años
   -    5: El ususario tiene entre 41 y 65 años
   -    6: El ususario tiene 66 años o más
 - zip_code: Texto que indica el código postal del ususario que ha realizado el movimiento
 - idunplug_station: Número de la estación de la que se desengancha la bicicleta.
 - idunplug_base: Número de la base en la que se desengancha la bicicleta
 - idplug_station: Número de la estación en la que se engancha la bicicleta
 - idplug_base: Número de la base en la que se engancha la bicicleta
 - unplug_hourTime: Franja horaria en la que se realiza el desenganche de la bicicleta.
 - travle_time: Tiempo total en segundos entre el desenganche y el enganche de la bicicleta.
 - track: Detalle del trallecto realizado por la bicicleta entre la estación de partida y la de destino

#### Para los archivos en formato csv
 - fecha: Día en el que se produce el movimiento 
 - idBike: Identificación del movimiento
 - fleet: Flota
 - trip_minutes: Tiempo total en minutos entre el desenganche y el enganche de la bicicleta
 - geolocation_unlock: Localización del desenganche
 - address_unlock: Dirección del desenganche
 - unlock_date: Fecha y hora en la que se realiza el desenganche
 - locktype y unclocktype: Lugar en la que se desengancha y encgancha la bicicleta
 - geolocation_lock: Localización del enganche
 - address_lock: Dirección del enganche
 - lock_date: Fecha y hora del enganche
 - station_unlock y station_lock: Número de estación en la que se engancha y desengancha la bicicleta
 - dock_unclock y dock_lock:  Número de base en la que se engancha y desengancha la bicicleta
 - unlock_station_name y lock_station_name: Nombre de la estación en la que se engancha y desengancha la bicicleta.




In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.getOrCreate()

23/05/27 12:42:27 WARN Utils: Your hostname, prpa resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/05/27 12:42:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/27 12:42:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
import json
import csv

Como hemos mencionado anteriormente, tendremos dos tipos de archivos, json y csv, que se ven de la siguiente manera:

In [20]:
enero2019 = spark.read.json('201901.json')
enero2022 = spark.read.csv('202201.csv', sep=';', header = True)

print('Archivo tipo json, enero 2019')
enero2019.show()
print('Archivo tipo csv, enero 2022')
enero2022.show()

Archivo tipo json, enero 2019
+--------------------+--------------------+--------+-----------+--------------+-------------+----------------+--------------------+-----------+--------------------+--------------------+---------+--------+
|     _corrupt_record|                 _id|ageRange|idplug_base|idplug_station|idunplug_base|idunplug_station|               track|travel_time|     unplug_hourTime|       user_day_code|user_type|zip_code|
+--------------------+--------------------+--------+-----------+--------------+-------------+----------------+--------------------+-----------+--------------------+--------------------+---------+--------+
|                null|{5c4b07ea2f38432e...|       0|         14|            83|           23|              82|{[{{[-3.6803143, ...|        162|{2019-01-01T00:00...|b3b5b61509ab89afb...|        1|        |
|                null|{5c4b07ea2f38432e...|       5|         10|            36|            1|             133|{[{{[-3.7063451, ...|        415|{2019-0

In [8]:
enero2019 = spark.read.json('201901.json')
enero2020 = spark.read.json('202001.json')
enero2021 = spark.read.json('202101.json')
enero2022 = spark.read.csv('202201.csv', sep=';', header = True)
enero2023 = spark.read.csv('202301.csv', sep=';', header = True)

In [30]:
print('Número de desplazamientos Enero 2019: ' + str(enero2019.count()))
print('Número de desplazamientos Enero 2020: ' + str(enero2020.count()))
print('Número de desplazamientos Enero 2021: ' + str(enero2021.count()))
print('Número de desplazamientos Enero 2022: ' + str(enero2022.count()//2))
print('Número de desplazamientos Enero 2023: ' + str(enero2023.count()//2))

Número de desplazamientos Enero 2019: 279223
Número de desplazamientos Enero 2020: 272875
Número de desplazamientos Enero 2021: 125642
Número de desplazamientos Enero 2022: 244760
Número de desplazamientos Enero 2023: 295923


El número de desplazamientos de enero de 2022 y 2023 lo hemos dividido entre dos porque los datos están intercalados con filas en blanco

Como podemos observar, en 2021 sufre una caída en la utilización de la bicicleta

Vamos a ver las estaciones donde más se desbloquean las bicicletas

In [10]:
unlock2019 = enero2019.groupBy('idunplug_station').count()
unlock2020 = enero2020.groupBy('idunplug_station').count()
unlock2021 = enero2021.groupBy('idunplug_station').count()
unlock2022 = enero2022.groupBy('station_unlock').count()
unlock2023 = enero2023.groupBy('station_unlock').count()


In [11]:
print ('ENERO 2019')
unlock2019.filter(unlock2019['count']>3400).show()
print ('ENERO 2020')
unlock2020.filter(unlock2020['count']>3000).show()
print ('ENERO 2021')
unlock2021.filter(unlock2021['count']>1100).show()
print ('ENERO 2022')
unlock2022.filter(unlock2022['count']>2300).show()
print ('ENERO 2023')
unlock2023.filter(unlock2023['count']>2500).show()

ENERO 2019


+----------------+-----+
|idunplug_station|count|
+----------------+-----+
|            null|47215|
|             175| 3480|
|             163| 3708|
|             129| 3513|
+----------------+-----+

ENERO 2020


+----------------+-----+
|idunplug_station|count|
+----------------+-----+
|              57| 3075|
|              43| 3674|
|              83| 3057|
|             175| 3740|
+----------------+-----+

ENERO 2021
+----------------+-----+
|idunplug_station|count|
+----------------+-----+
|              57| 1637|
|              43| 1586|
|             208| 1158|
|             175| 1182|
+----------------+-----+

ENERO 2022


+--------------+------+
|station_unlock| count|
+--------------+------+
|          null|246630|
|            43|  2846|
|           175|  2421|
|            57|  2377|
+--------------+------+

ENERO 2023


+--------------+------+
|station_unlock| count|
+--------------+------+
|          null|296714|
|            43|  3657|
|           220|  2671|
|           175|  2832|
|            57|  3752|
+--------------+------+



#### ESTACIONES DONDE MÁS SE COGEN BICICLETAS

ENERO 2019
- 163: Paseo de la Esperanza, 83
- 129: Calle de Jose Abascal, 33
- 175: Calle Jaime el Conquistador, 30

ENERO 2020
- 175: Calle Jaime el Conquistador, 30
- 43: Calle de Lavapiés, 36
- 57/83: Calle de Santa Cruz de Marcenado, 24 / Pio Baroja, 8B

ENERO 2021
- 57: Calle de Santa Cruz de Marcenado, 24
- 43: Calle de Lavapiés, 36
- 175/208: Calle Jaime el Conquistador, 30 / Paseo de la Castellana, S/N

ENERO 2022
- 43: Calle de Lavapiés, 36
- 175: Calle Jaime el Conquistador, 30
- 57: Calle de Santa Cruz de Marcenado, 24

ENERO 2023
- 57: Calle de Santa Cruz de Marcenado, 24
- 43: Calle de Lavapiés, 36
- 175: Calle Jaime el Conquistador, 30

En general, donde más se cogen es en la estación 175, 43, 57

Ahora, vamos a ver las estaciones donde más se bloquean las bicicletas

In [12]:
lock2019 = enero2019.groupBy('idplug_station').count()
lock2020 = enero2020.groupBy('idplug_station').count()
lock2021 = enero2021.groupBy('idplug_station').count()
lock2022 = enero2022.groupBy('station_lock').count()
lock2023 = enero2023.groupBy('station_lock').count()

In [13]:
print ('ENERO 2019')
lock2019.filter(lock2019['count']>3300).show()
print ('ENERO 2020')
lock2020.filter(lock2020['count']>3000).show()
print ('ENERO 2021')
lock2021.filter(lock2021['count']>1100).show()
print ('ENERO 2022')
lock2022.filter(lock2022['count']>2300).show()
print ('ENERO 2023')
lock2023.filter(lock2023['count']>2500).show()

ENERO 2019


+--------------+-----+
|idplug_station|count|
+--------------+-----+
|            43| 3336|
|          null|47215|
|           175| 3608|
|           163| 3774|
|           129| 3396|
+--------------+-----+

ENERO 2020


+--------------+-----+
|idplug_station|count|
+--------------+-----+
|            57| 3057|
|            43| 3748|
|            83| 3068|
|           175| 3741|
+--------------+-----+

ENERO 2021
+--------------+-----+
|idplug_station|count|
+--------------+-----+
|           149| 1101|
|            57| 1639|
|            43| 1594|
|           208| 1179|
|           175| 1170|
+--------------+-----+

ENERO 2022


+------------+------+
|station_lock| count|
+------------+------+
|        null|247067|
|          43|  2901|
|         175|  2508|
|          57|  2418|
+------------+------+

ENERO 2023


+------------+------+
|station_lock| count|
+------------+------+
|        null|296887|
|          43|  3698|
|         220|  2848|
|         175|  2906|
|          57|  3759|
+------------+------+



#### ESTACIONES DONDE MÁS SE DEVUELVEN BICICLETAS

ENERO 2019
- 163: Paseo de la Esperanza, 83
- 175: Calle Jaime el Conquistador, 30
- 129/43: Calle Jose Abascal, 33 / Calle de Lavapiés, 36

ENERO 2020
- 43: Calle de Lavapiés, 36
- 175: Calle Jaime el Conquistador, 30
- 83/57: Pio Baroja, 8B / Calle de Santa Cruz de Marcenado, 24

ENERO 2021
- 57: Calle de Santa Cruz de Marcenado, 24
- 43: Calle de Lavapiés, 36
- 208/175: Paseo de la Castellana, S/N / Calle Jaime el Conquistador, 30

ENERO 2022
- 43: Calle de Lavapiés, 36
- 175: Calle Jaime el Conquistador, 30
- 57: Calle de Santa Cruz de Marcenado, 24

ENERO 2023
- 57: Calle de Santa Cruz de Marcenado, 24
- 43: Calle de Lavapiés, 36
- 175/220: Calle Jaime el Conquistador, 30 / Arroyo de las Pilillas , 4

En general, donde más se devuelven es en la estación 175, 43, 57

Se podría decir que las estaciones más utilizadas son 175, 43, 57

A continuación, veremos las estaciones donde menos bicicletas se desbloquean

In [20]:
print ('ENERO 2019')
unlock2019.filter(unlock2019['count']<350).show()
print ('ENERO 2020')
unlock2020.filter(unlock2020['count']<400).show()
print ('ENERO 2021')
unlock2021.filter(unlock2021['count']<60).show()
print ('ENERO 2022')
unlock2022.filter(unlock2022['count']<20).show()
print ('ENERO 2023')
unlock2023.filter(unlock2023['count']<200).show()

ENERO 2019


+----------------+-----+
|idunplug_station|count|
+----------------+-----+
|              29|  327|
|              28|  268|
|            2008|    1|
+----------------+-----+

ENERO 2020


+----------------+-----+
|idunplug_station|count|
+----------------+-----+
|              29|  339|
|              28|  167|
+----------------+-----+

ENERO 2021
+----------------+-----+
|idunplug_station|count|
+----------------+-----+
|             257|   49|
|             265|   52|
+----------------+-----+

ENERO 2022
+--------------+-----+
|station_unlock|count|
+--------------+-----+
|           115|   11|
|           123|    9|
|           124|   12|
+--------------+-----+

ENERO 2023


+--------------+-----+
|station_unlock|count|
+--------------+-----+
|           117|  164|
|           889|    3|
|            89|    9|
+--------------+-----+




[Stage 77:===========================================>              (3 + 1) / 4]



#### ESTACIONES DONDE MENOS SE COGEN BICICLETAS

ENERO 2019
- 2008: NO EXISTE
- 28: Calle de Alcalá, 26
- 29: Marqués de Cubas, 25

ENERO 2020
- 28: Calle de Alcalá, 26
- 29: Marqués de Cubas, 25

ENERO 2021
- 49: Ronda de Atocha, 36
- 52: Plaza de Santa Ana, 9

ENERO 2022
- 123: Guzmán el Bueno, 2
- 115/124: Plaza del Marqués de Cerralbo, S/N / Manuel Silvela, 18

ENERO 2023
- 889: NO EXISTE
- 89: Ayala, 102
- 117: Altamirano, 24

Podemos ver una diferencia muy grande entre estaciones, siendo la 89 utilizada 9 veces y la 117 utilizada 164 veces

Ninguna de las estaciones con menor número de desengaches se repite a lo largo de los años.

También podemos ver que algunos datos se han recogido erroneamente como en enero de 2019 y 2023, donde aparecen estaciones que no existen

Ahora veremos las estaciones donde menos se bloquean las bicicletas

In [14]:
print ('ENERO 2019')
lock2019.filter(lock2019['count']<350).show()
print ('ENERO 2020')
lock2020.filter(lock2020['count']<400).show()
print ('ENERO 2021')
lock2021.filter(lock2021['count']<60).show()
print ('ENERO 2022')
lock2022.filter(lock2022['count']<200).show()
print ('ENERO 2023')
lock2023.filter(lock2023['count']<200).show()

ENERO 2019


+--------------+-----+
|idplug_station|count|
+--------------+-----+
|            29|  344|
|            28|  271|
|          2008|    1|
+--------------+-----+

ENERO 2020


+--------------+-----+
|idplug_station|count|
+--------------+-----+
|            29|  332|
|            28|  170|
+--------------+-----+

ENERO 2021
+--------------+-----+
|idplug_station|count|
+--------------+-----+
|           266|   52|
|           257|   35|
|           265|   52|
|           267|   59|
|           261|   54|
+--------------+-----+

ENERO 2022


+------------+-----+
|station_lock|count|
+------------+-----+
|          28|  185|
|         257|  183|
|         266|  117|
|         225|  168|
+------------+-----+

ENERO 2023


+------------+-----+
|station_lock|count|
+------------+-----+
|          28|  180|
|         117|  160|
|         889|    4|
|          89|   12|
+------------+-----+



#### ESTACIONES DONDE MENOS SE DEVUELVEN BICICLETAS

ENERO 2019
- 2008: NO EXISTE
- 28: Calle de Alcalá, 26
- 29: Marqués de Cubas, 25

ENERO 2020
- 28: Calle de Alcalá, 26
- 29: Marqués de Cubas, 25

ENERO 2021
- 257: Avenida de Juan de Herrera, 1
- 266/265: NO APARECEN EN LA WEB

ENERO 2022                                                                           
- 266: NO APARECE EN LA WEB
- 225: Sierra Toledana, 33

La 266 sigue abajo en el ranking pero ha aumentado considerablemente su uso


ENERO 2023
- 889: NO EXISTE
- 89: Ayala, 102
- 117/28: Altamirano, 24 / Alcalá , 26

En general, donde menos se devuelven es en la estación 28

Una vez más, podemos ver que aparecen datos desactualizados en 2021 y 2022, y algunos erróneos en 2019 y 2023

Se podría decir que la estación menos utilizada es la 28

Vamos a separar por tipos de archivos para poder tratar mejor los datos, empezamos con los de tipo json y finalizamos con los de tipo csv

#### ARCHIVOS JSON (2019, 2020, 2021)

Analizamos el tipo de usuario que más utiliza el servicio

In [23]:
print('TIPO DE USUARIO QUE LO UTILIZA')
print ('ENERO 2019')
enero2019.groupBy('user_type').count().show()
print ('ENERO 2020')
enero2020.groupBy('user_type').count().show()
print ('ENERO 2021')
enero2021.groupBy('user_type').count().show()

TIPO DE USUARIO QUE LO UTILIZA
ENERO 2019


+---------+------+
|user_type| count|
+---------+------+
|        0|  1323|
|     null| 47215|
|        1|210211|
|        3| 19069|
|        2|  1405|
+---------+------+

ENERO 2020


+---------+------+
|user_type| count|
+---------+------+
|        1|249537|
|        3| 22693|
|        2|   645|
+---------+------+

ENERO 2021
+---------+------+
|user_type| count|
+---------+------+
|        6|   684|
|        1|119912|
|        3|  4762|
|        2|   284|
+---------+------+



#### TIPO DE USUARIO QUE MÁS UTILIZA EL SERVICIO
El grupo de usuarios que más utiliza la bicicleta es el número 1 y 3, asociados a pase anual y empleados.
En enero 2021 desciende considerablemente el número, como hemos visto antes.

Ahora, observemos el tiempo de utilización total de las bicicletas por tipo de usuario

In [29]:
print('TIEMPO EN SEGUNDOS UTILIZACIÓN TOTAL')
print ('ENERO 2019')
enero2019.groupBy('user_type').sum('travel_time').show()
print ('ENERO 2020')
enero2020.groupBy('user_type').sum('travel_time').show()
print ('ENERO 2021')
enero2021.groupBy('user_type').sum('travel_time').show()

TIEMPO EN SEGUNDOS UTILIZACIÓN TOTAL
ENERO 2019


+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        0|         2592405|
|     null|            null|
|        1|       165544356|
|        3|        69331926|
|        2|         2563298|
+---------+----------------+

ENERO 2020


+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        1|       211759904|
|        3|        92907755|
|        2|         1505971|
+---------+----------------+

ENERO 2021
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        6|         1351215|
|        1|       126578813|
|        3|        27008414|
|        2|          699819|
+---------+----------------+



#### TIEMPO DE UTILIZACIÓN MAYOR POR GRUPO

Como podemos observar, el usuario con pase anual es el que más lo utiliza, seguido siempre del trabajador de empresa y por último, turista. Menos en 2021, que aparece el dato de usuario ocasional que ocupa el tercer puesto y el usuario turista queda último

Una vez visto los datos, también podemos comprobar que el tiempo de utilización de enero 2021 fue mucho menor, como podíamos suponer


A continuación, veremos las veces que se utiliza el servicio según el rango de edad

In [17]:
print('RANGO DE EDAD UTILIZADO')
print ('ENERO 2019')
enero2019.groupBy('ageRange').count().show()
print ('ENERO 2020')
enero2020.groupBy('ageRange').count().show()
print ('ENERO 2021')
enero2021.groupBy('ageRange').count().show()


RANGO DE EDAD UTILIZADO
ENERO 2019


+--------+-----+
|ageRange|count|
+--------+-----+
|       0|94940|
|    null|47215|
|       6|  916|
|       5|49893|
|       1| 2662|
|       3|13067|
|       2| 1600|
|       4|68930|
+--------+-----+

ENERO 2020
+--------+------+
|ageRange| count|
+--------+------+
|       0|118379|
|       6|  1441|
|       5| 61718|
|       1|  2993|
|       3| 11869|
|       2|  1148|
|       4| 75327|
+--------+------+

ENERO 2021
+--------+-----+
|ageRange|count|
+--------+-----+
|       0|67842|
|       6|  625|
|       5|23925|
|       1| 1373|
|       3| 4910|
|       2|  564|
|       4|26403|
+--------+-----+



#### RANGO DE EDAD MÁS UTILIZADO

El grupo de rango de edad más utilizado que se pudo determinar suele ser siempre el 4 y 5, usuarios entre 27 y 65 años.
Luego, le sigue el 3 (19-26).
Quedando como menos utilizados el grupo 1,2 y 6

- 0: No se ha podido determinar el rango de edad del usuario
- 1: El usuario tiene entre 0 y 16 años
- 2: El usuario tiene entre 17 y 18 años
- 3: El usuario tiene entre 19 y 26 años
- 4: El usuario tiene entre 27 y 40 años
- 5: El usuario tiene entre 41 y 65 años
- 6: El usuario tiene 66 años o más

#### ARCHIVOS CSV (2022, 2023)

Veamos la fecha cuando más se utilizó el servicio

In [18]:
print('FECHA MÁS UTILIZADAS DE LOS ÚLTIMOS 2 AÑOS')

fecha2022 = enero2022.groupBy('fecha').count()
fecha2023 = enero2023.groupBy('fecha').count()

print ('ENERO 2022')
fecha2022.filter(fecha2022['count']>10000).show()
print ('ENERO 2023')
fecha2023.filter(fecha2023['count']>12000).show()

FECHA MÁS UTILIZADAS DE LOS ÚLTIMOS 2 AÑOS
ENERO 2022


+----------+------+
|     fecha| count|
+----------+------+
|      null|244760|
|2022-01-18| 10228|
|2022-01-13| 10237|
|2022-01-20| 10025|
|2022-01-26| 10154|
|2022-01-28| 10088|
|2022-01-27| 10511|
+----------+------+

ENERO 2023


+----------+------+
|     fecha| count|
+----------+------+
|      null|295923|
|2023-01-10| 12613|
|2023-01-12| 13362|
|2023-01-13| 12829|
|2023-01-11| 12304|
+----------+------+



#### FECHA CUANDO MÁS SE UTILIZA

Como podemos ver:

Los días cuando bicimad se utilizó más de enero de 2022 son: 27, 13, 18

Los días cuando bicimad se utilizó más de enero de 2023 son: 12, 13, 10

Podemos intuir que es debido al fin de las vacaciones de Navidad y la vuelta al trabajo.

# Pandemia

¿Cómo afectó la pandemia al uso de bicimad?¿Se utilizó más una vez terminado el confinamiento en comparación a los meses previos?

Veamos el número de desplazamientos antes, durante y post confinamiento 2020

In [19]:
febrero2020 = spark.read.json('202002.json')
marzo2020 = spark.read.json('202003.json')
abril2020 = spark.read.json('202004.json')
mayo2020 = spark.read.json('202005.json')
junio2020 = spark.read.json('202006.json')

print('Número de desplazamientos Febrero 2020: ' + str(febrero2020.count()))
print('Número de desplazamientos Marzo 2020: ' + str(marzo2020.count()))
print('Número de desplazamientos Abril 2020: ' + str(abril2020.count()))
print('Número de desplazamientos Mayo 2020: ' + str(mayo2020.count()))
print('Número de desplazamientos Junio 2020: ' + str(junio2020.count()))

Número de desplazamientos Febrero 2020: 341972
Número de desplazamientos Marzo 2020: 148262
Número de desplazamientos Abril 2020: 10447
Número de desplazamientos Mayo 2020: 259579
Número de desplazamientos Junio 2020: 210555


Abril 2020 se podría decir que es uno de los meses en los que menos se ha utilizado el servicio desde su comienzo

# CONCLUSIÓN

##### Con los datos que se han recopilado se pueden obtener varias conclusiones.
##### En cuanto a la evolución que se ha producido en el número de desplazamientos a lo largo de los años se puede afirmar que el uso de este servicio se ha mantenido bastante constante. Por lo general, la cifra de desplazamientos en el mes de enero oscila entre los 250000 y los 300000. La única cifra que se aleja considerablemente del resto de datos es la de enero de 2021, que estaría altamente relacionada con el temporal de Filomena que azotó la ciudad.
##### Las estaciones más utilizadas se encuentran cerca de Madrid Río, por la parte de Chopera, Acacias y Usera. Podemos asociarlo a la habilitación de carriles bici que se encuentran ahí, además de ser una zona agradable para montar en bicicleta. La menos, sin embargo, se encuentra en San Bernardo, pleno corazón de la ciudad, donde no hay carriles bici y es peligroso por el tráfico. También se encuentra mal posicionada justo al lado de una boca de metro.
##### Respecto a los usuarios más frecuentes del servicio cabe destacar que para la mayor parte de los datos no se ha podido determinar ni su rango de edad ni el grupo al que pertenecen. Sin embargo, para los resultados que sí se han podido recoger se puede concluir que las personas que más utilizan el servicio, tanto en trayecto como en tiempo, son los usuarios de tipo 1 y 3 asociados a pase anual y empleados, mientras que el grupo de rango de edad que más lo utiliza es aquel comprendido entre los 27 y los 65. Son los más jóvenes y los mayores quienes menos utilizan esta forma de transporte. 
##### Con relación a los días del mes de enero cuando más se utiliza el servicio en los dos últimos años se puede relacionar con el fin de las vacaciones de Navidad y la vuelta al trabajo. También podría deberse al afán por comenzar el año de manera saludable
##### Finalmente, en las cifras recogidas en los primeros meses del año 2020 se puede percibir el cambio radical que se produce con el inicio de la pandemia. Se ve como de febrero a marzo se produce una disminución considerable en el número de desplazamientos con, casi, 200000 deplazamientos menos de un mes a otro. Esto coincide con la declaración del estado de alarma que se produce el 14 de marzo. Por otra parte, el dato más significativo es el que se recoge en el mes de abril con una cifra de desplazamientos insignificante debido al confinamiento de la población durante ese mes. Posteriormente se percibe el cambio en el modo de comportamiento de la población con un aumento de más de 250000 desplazamientos en el mes de mayo.